In [1]:
%run ..\notebooks\Util_func.ipynb

In [2]:
# # Bay Area
# REG = 'BayArea'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\spatial_join'

# out_file = r'out\%s_10_TNCTrips_Cost.xlsx' %REG

In [3]:
# # SANDAG
# REG = 'SANDAG'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG'

# out_file = r'out2\%s_10_TNCTrips_Cost.xlsx' %REG

In [4]:
# SCAG
REG = 'SCAG'
base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27\2_tour_extract\wt_wkday'
raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27'

out_file = r'out2\%s_10_TNCTrips_Cost.xlsx' %REG

In [5]:
## Process person records
tmp_df = pd.read_csv(join(raw_dir, 'ex_person_wZones.csv'))
if REG=='SANDAG' or REG=='SCAG':
    tmp_df2 = pd.read_csv(join(raw_dir, 'ex2_person.tsv'), sep='\t')
    tmp_df = tmp_df.merge(tmp_df2[['hh_id','person_id','raceeth_new_imputed']], how='left')
tmp_df = tmp_df[['hh_id','person_num','raceeth_new_imputed','income_imputed','gender','age']]
tmp_df = tmp_df.rename(columns={'raceeth_new_imputed':'raceeth','income_imputed':'hinc'})
tmp_df = tmp_df.rename(columns={'hh_id':'hhno','person_num':'pno'})

per_df = pd.read_csv(join(base_dir,'survey2018_precx.dat'), sep=' ') 
per_df = per_df.merge(tmp_df, how='left')

## Process trip records
raw_trips = pd.read_csv(join(raw_dir, 'ex_trip_wZones.csv'))
raw_trips = raw_trips[['hh_id','person_num','trip_num','taxi_cost','mode_uber','mode_lyft','mode_type_imputed']]
raw_trips = raw_trips.rename(columns={'hh_id':'hhno','person_num':'pno','trip_num':'tsvid'})

trip_df = pd.read_csv(join(base_dir,'survey2018_tripx.dat'), sep=' ')
trip_df = link_dt(trip_df)
trip_df = trip_df.loc[trip_df['mode']==9, ]
trip_df = trip_df.merge(raw_trips, how='left')
trip_df = trip_df[trip_df['mode_type_imputed']!=4] #remove taxi trips

trip_df['tnc_type'] = 3 # prem/other
trip_df.loc[(trip_df['mode_uber']==1) | (trip_df['mode_lyft']==1), 'tnc_type'] = 1 # pooled
trip_df.loc[(trip_df['mode_uber']==2) | (trip_df['mode_lyft']==2), 'tnc_type'] = 2 # regular

trip_df = trip_df.merge(per_df[['hhno','pno','raceeth','hinc','gender','age']], 
                        how='left', on=['hhno','pno'])
trip_df.loc[trip_df['gender']==997, 'gender'] = 5 #Other
trip_df.loc[trip_df['gender']==999, 'gender'] = 6 #NoAnswer
trip_df.loc[trip_df['gender'].isin([-9998, 995]), 'gender'] = 6 #Missing

trip_df = trip_df[(trip_df['trexpfac']>0)]
trip_df = trip_df[(trip_df['otaz']>0) & (trip_df['dtaz']>0)]
trip_df['count'] = 1
trip_df = trip_df[(trip_df['taxi_cost']>0)]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [6]:
col_dict = {
    'raceeth': {
        'desc': 'RaceEth',
        'col': 'raceeth',
        'vals': range(1,6),
        'labels': ['1_Hispanic','2_Black','3_Asian/PI','4_White','5_Other']
    },
    'hinc': {
        'desc': 'HHInc',
        'col': 'hinc',
        'vals': range(1,9),
        'labels': ['1_25K','2_25_50K','3_50_75K','4_75_100K','5_100_150K','6_150_200K','7_200_250K','8_250K']
    },
    'age': {
        'desc': 'Age',
        'col': 'age',
        'vals': range(4,11),
        'labels': ['18-24','25-34','35-44','45-54','55-64','65-74','75+']
    },
    'gender': {
        'desc': 'Gend',
        'col': 'gender',
        'vals': range(1,7),
        'labels': ['1_F','2_M','3_Trns','4_NBin','5_Oth','6_Miss']
    },
    'tnc_freq': {
        'desc': 'TNCFreq',
        'col': 'tnc_freq',
        'vals': list(range(1,8)) + [995],
        'labels': ['1_6-7dpwk','2_5dpwk','3_4dpwk','4_2-3dpwk','5_1dpwk','6_1-3dpmo','7_0-1dpmo','8_Never']
    },
    'tncmode': {
        'desc': 'TNCMode',
        'col': 'tnc_type',
        'vals': range(1,4),
        'labels': ['2_Pool','1_Reg','3_PremOth']
    }
}

In [7]:
agg_col = 'taxi_cost'
def agg_tnc(df, colname, cat_levels, cat_labels):
    cat_levels = list(cat_levels) + ['All']
    cat_labels = cat_labels + ['All']
    base_df = pd.DataFrame({'cat_labels': cat_labels, colname: cat_levels})
    
    df = df[[colname,'count','trexpfac',agg_col]]
    df['wtd_trips'] = df['count']*df['trexpfac']
    df['wtd_metric'] = df[agg_col]*df['trexpfac']
    df = pd.DataFrame(pd.pivot_table(df, index=colname, margins=True, aggfunc=np.sum)).reset_index()
    df = base_df.merge(df, how='left')
    df = df.fillna(0)
    df['sample_avg'] = df[agg_col]/df['count']
    df['wtd_avg'] = df['wtd_metric']/df['wtd_trips']
    
    df = df.drop([colname], axis=1)
    df = df.rename(columns={'cat_labels':colname})
    df = df[[colname,'sample_avg','wtd_avg']].set_index(colname)
    return df

In [8]:
fname = out_file
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '#,##0.0'})

In [9]:
from xlsxwriter.utility import xl_rowcol_to_cell

PREF = 'Weekday Avg TNC Cost by '
for key in ['raceeth', 'hinc', 'age', 'gender', 'tncmode']:
    d1_dict = col_dict[key]
    row = 0
    sname = d1_dict['desc']
    title = PREF + d1_dict['desc']
    tab = agg_tnc(trip_df, d1_dict['col'], d1_dict['vals'], d1_dict['labels'])
    row = write_to_excel(tab, sname, title, row)
    
    tab_range = xl_rowcol_to_cell(row,1) + ':' + xl_rowcol_to_cell(row,tab.shape[1])
    writer.sheets[sname].set_column(tab_range, 18, format1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [10]:
writer.save()